In [1]:
import pandas as pd
import re
import sys
import numpy as np
import nltk
from nltk.corpus import wordnet as wn
from nltk import pos_tag
from nltk.tag import UnigramTagger
from nltk.corpus import brown
import webcolors
from IPython.display import display
sys.path.append("..")
from ADA_JEX2017.Project.Functions.str_functions import *
from ADA_JEX2017.Project.Functions.pre_process import *
import math


In [2]:
path='../ADA_JEX2017/Project/Functions/'

In [3]:
#our datafile exported to csv
data_file='./recipeInfo/recipeInfo_WestWhiteHorvitz_WWW2013.csv'
raw_data = pd.read_csv(data_file ,sep=';')

In [4]:
with open(path+'units_list.txt', 'r') as f:
    units_list = [line.rstrip('\n') for line in f]
    
with open(path+'technique_list.txt', 'r') as f:
    techniques_list = [line.rstrip('\n') for line in f]

with open(path+'ingredient_list.txt', 'r') as f:
    ingredient_list = [line.rstrip('\n') for line in f]
    
lemmatizer = WordNetLemmatizer()
ingredient_list=[lemmatizer.lemmatize(token).lower() for token in ingredient_list]

In [5]:
ingr_dataframe=raw_data[['title','ingredients_list','ingredients_bag-of-words']].copy().dropna()
ingr_dataframe = ingr_dataframe.reset_index(drop=True)
display(ingr_dataframe.head())
ingr_data_reduced=ingr_dataframe.head(100)
#we ignore upper case in the ingredients string
ingr_dataframe['ingredients_list']=ingr_dataframe['ingredients_list'].str.lower()

,title,ingredients_list,ingredients_bag-of-words
0,Easy Light Chocolate Milkshake Recipe,"put one half cup of milk, 4 tablespoons of cho...","! , . 4 ? and are blend chocolate cup enjoy ex..."
1,Lamb Stew Recipe : : Recipes : Food Network,5 yellow onions|2 turnips|5 carrots|1 stalk fe...,", 1 1\/2 1\/4 1\/8 1Â 1\/2 2 3 5 and anise app..."
2,Chocolate Bread Pudding Recipe : Paula Deen : ...,"1 (1-pound) loaf French or Italian bread, cube...",", -lrb- -rrb- 1 1-pound 1\/2 1\/4 1Â 1\/2 2 3 ..."
3,Snowball Cookies II Recipe,1/2 cup powdered sugar|1/3 cup butter or marga...,1 1\/2 1\/3 2 almond baking bisquick butter ch...
4,Old Fashioned Butterscotch Pie Recipe #31698,1 1/2 cup brown sugar|1 cup water|3 eggs|4 tab...,1 1\/2 1\/4 1Â 1\/2 3 4 brown butter cream cup...


In [6]:
fun_add_preprocess(ingr_dataframe,units_list)

In [32]:
def fun_covert_to_float(frac_str):

    splited_frac_str = frac_str.split(' ')

    if '/' in frac_str :
        fractionSplit = splited_frac_str[0].split('/')
        frac = float(fractionSplit[0])/float(fractionSplit[1])
        return frac
    elif bool(re.search('\.([0-9])\.',frac_str)):
        return float(re.findall('\.([0-9])\.',frac_str)[0])
    else:
        frac = float(splited_frac_str[0])
        return frac

def fun_unit_corrector(string):

    newUnit = ""
    splited_string = string.split(' ')
    dict_conver = {'teaspoon' : 5,'tablespoon' :15, 'oz': 30,'cup': 237,'pint' : 474,'quart':946, 'gallon': 3785,'dl' :100,'pound' :453,'ounce' :29,'g' : 1,'kg': 1000,  \
        'l': 1000, 'ml': 1,'mg':1,'bottle':750,'drop':0.05,'pinch':0.36,'jar':1000,'can':330,'unit':'u','gill':118}

    teaspoon = ['teaspoon','tsp','t','teaspoon']
    tablespoon = ['tablespoon' ,'T', 'tbl', 'tbs','tbsp']
    oz = ['oz', 'fl', 'fluid ounce']
    cup = ['cup', 'c']
    pint =  ['pint','p', 'pt', 'fl pt']
    quart =  ['quart','q', 'qt','fl qt']
    gallon =  ['gallon', 'gal']
    ml = ['ml', 'milliliter', 'millilitre', 'cc','mL']
    l = ['l', 'liter', 'litre', 'L']
    dl = ['dl','deciliter','decilitre','dL']
    pound = ['pound','lb']
    ounce = ['ounce','oz']
    mg = ['mg','milligram','milligramme']
    kg = ['kg', 'kilogramme','kilogram']
    g = ['g','gram']
    unit = ['unit','stalk','package']
    other_indic=['bottle','pinch','jar','can','drop','gill']

    unit_list = [teaspoon, tablespoon, oz, cup, pint, pint, quart, gallon, ml, l, dl, pound, ounce, mg, kg, g]

    #if (len(splited_string) == 1 and splited_string[0] == ''):
    #    return ''

    for lists in unit_list:
        for i,x in enumerate(splited_string):
            if x in lists:
                newUnit = lists[0]
                gram_value = float(dict_conver[newUnit]) * fun_covert_to_float(splited_string[abs(i-1)])
                return gram_value
            if x in other_indic:
                gram_value = float(dict_conver[x]) * fun_covert_to_float(splited_string[abs(i-1)])
                return gram_value
            if x in unit:
                return splited_string[abs(i-1)]+' u'

    return 0


def fun_extract_ingredients(one_receipe,ingredients_list,techniques_list,units_list,to_gram=True):
    ''' Function extractiing all ingredients, quantities and possiblity technics of cooking

    '''
    lemmatizer = WordNetLemmatizer()
    if '|' in one_receipe:
        ingredients=one_receipe.split('|')
    else:
        ingredients=one_receipe.split(', ')

    dic_ingre={}
    dic_tec={}
    wasted_ingr=[]
    wasted_number=0
    for elem in ingredients:
        #split in words
        elem=elem.replace('-',' ')
        elem_list=elem.split(' ')
        #avoid special characters appearing in some recipes:
        if '&#' in elem:
            continue
        #keep only alphanumerics in each words
        elem_list=[re.sub('[^0-9a-zA-Z/. ]+', '', x) for x in elem_list]
        #keep only the root of the word
        check = [lemmatizer.lemmatize(token) for token in elem_list]
        #split str of string with stuck digit : '2cups': '2','cups'
        check=sum([re.findall(r'[A-Za-z]+|[\d.]+', x) for x in check],[])
        techniques=[]
        units=[]
        one_ingr=None
        no_unit=True
        no_number=True
        check = list(filter(None, check))
        for word in check:
            if word in techniques_list:#check if it belongs to our technics list
                techniques.append(word)
            elif word in ingredients_list:#check if it belongs to our ingredient list
                one_ingr=word
            elif bool(re.search(r'\d',word)) and (no_number):#check if it belongs to our unit list or is alphanumeric
                units.append(word)
                no_number=False
            elif (word in units_list) and (no_unit):
                units.append(word)
                no_unit=False
        for biword in nltk.bigrams(check): # check if we have a biword ingredient
            if ' '.join(biword) in ingredients_list:
                one_ingr=' '.join(biword)
        if one_ingr==None :      # check if we have no ingredient : avoid this element of recipe
            wasted_number=wasted_number+1
            wasted_ingr.append(' '.join(check))
            continue
        if(len(' '.join(units))==0):  # fill with a special unit if we are dealing with no quantity
            units.append('1')
            units.append('unit')
        elif no_unit:
            units.append('unit')
        elif no_number:
            units.append('1')


        units=' '.join(units)
        if to_gram:
            units=fun_unit_corrector(units)

        dic_ingre[one_ingr]=units
        dic_tec[one_ingr]=' '.join(techniques)

    return dic_ingre,dic_tec,wasted_ingr,wasted_number

In [47]:
#-----test cell : test a specific receipe -------------
receipe=ingr_dataframe.loc[42577]['Recipe_preporcess']
print(receipe)
dic_ingr,dictec,wasted,wasted_numb=fun_extract_ingredients\
    (receipe,ingredient_list,techniques_list,units_list,to_gram=True)
dic_ingr

1 cup sugar|12 cup oleo|1 cup milk|1 eggs|1 tsp nutmeg|1 12 cups flour|18 tsp salt|2 tsps baking powder|12 tsp cinnamon|34 cup wheat germ|3 tbsps molasses|12 cup elderberry 


{'baking powder': '2 u',
 'cinnamon': 60.0,
 'egg': '1 u',
 'elderberry': 2844.0,
 'flour': 237.0,
 'milk': 237.0,
 'molasses': '3 u',
 'nutmeg': 5.0,
 'salt': 90.0,
 'sugar': 237.0}

In [ ]:
str_checked=[]
for x in ['2poulet','cup','1','.25']:
        if len(x)>1 :
            str_checked.append(sum([re.findall(r'[A-Za-z]+|[\d+.]', x)],[]))
        else: str_checked.append(x)
str_checked

In [26]:
re.findall(r'\.([0-9])\.', '0.26')

In [23]:
try:
    float('.1.')
except ValueError:
    return 0

SyntaxError: 'return' outside function (<ipython-input-23-0c71bcc2f456>, line 4)

In [34]:
#----- calculate whole data frame ---------
ingr_data_reduced=ingr_dataframe.head(100)
all_dic=[]
not_used_ingr=[]
wastes=0
for index, row in ingr_dataframe.iterrows():
    recipe=row['Recipe_preporcess']
    dic_ingre,dictec,wasted_ingr,wasted_number=fun_extract_ingredients\
            (recipe,ingredient_list,techniques_list,units_list,to_gram=True)
    all_dic.append(dic_ingre)
    not_used_ingr.append(wasted_ingr)
    wastes=wastes+wasted_number

In [131]:
# --- actually create the dataframe
ingredients_frame=pd.DataFrame(data=all_dic)
display(ingredients_frame.head(5))

,acidulated water,acorn squash,agave,ale,aleppo pepper,allspice,almond,almond extract,almond milk,amaranth,...,worcestershire sauce,wrap,yam,yeast,yellow lentil,yoghurt,yogurt,yuzu,zest,zucchini
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,237,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [58]:
sum((ingredients_frame==0).sum(axis=1))

0

In [134]:
ratio_u_ingred={}
test_ingredients_frame=ingredients_frame.copy()
for ingred in list(ingredients_frame):
    mean = ingredients_frame[ingred].apply(pd.to_numeric, errors='coerce').dropna(axis=0, how='any').mean()
    if math.isnan(mean):
        mean=1
    test_ingredients_frame[ingred][ingredients_frame[ingred].astype(str).str.contains('u')]=mean
    ratio_u_ingred[ingred]=\
    sum(ingredients_frame[ingred].dropna(axis=0, how='any').astype(str).str.contains('u'))/len(ingredients_frame[ingred].dropna(axis=0, how='any').astype(str).str.contains('u'))

//anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [135]:
display(test_ingredients_frame)

,acidulated water,acorn squash,agave,ale,aleppo pepper,allspice,almond,almond extract,almond milk,amaranth,...,worcestershire sauce,wrap,yam,yeast,yellow lentil,yoghurt,yogurt,yuzu,zest,zucchini
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,237,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [130]:
#ingredients_frame['acidulated water'].dropna(axis=0, how='any').mean()
import math
mean = ingredients_frame['acidulated water'].apply(pd.to_numeric, errors='coerce').dropna(axis=0, how='any').mean()
if math.isnan(2):
    print(mean)
s=ingredients_frame['acidulated water']
display(s)
s[ingredients_frame['acidulated water'].astype(str).str.contains('u')]=2
display(s)

0            NaN
1        705.205
2        333.861
3        302.155
4        333.861
5            NaN
6        9.81666
7        267.693
8            NaN
9            NaN
10       333.861
11       103.348
12       854.472
13           NaN
14       103.348
15       103.348
16       534.398
17       433.183
18       430.122
19           NaN
20       243.606
21       361.495
22       169.367
23           NaN
24           NaN
25       75.5559
26       13.2721
27       433.183
28       774.517
29       243.606
          ...   
46247    243.606
46248    433.183
46249    13.2721
46250    507.651
46251    783.828
46252    599.763
46253        NaN
46254    864.589
46255    333.861
46256    23.1258
46257    103.348
46258        NaN
46259    583.995
46260    228.447
46261        NaN
46262    265.357
46263    662.876
46264        NaN
46265    377.483
46266    484.098
46267    771.571
46268    333.861
46269        NaN
46270    1736.51
46271    18.8225
46272    302.155
46273    518.592
46274    333.8

//anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


0            NaN
1        705.205
2        333.861
3        302.155
4        333.861
5            NaN
6        9.81666
7        267.693
8            NaN
9            NaN
10       333.861
11       103.348
12       854.472
13           NaN
14       103.348
15       103.348
16       534.398
17       433.183
18       430.122
19           NaN
20       243.606
21       361.495
22       169.367
23           NaN
24           NaN
25       75.5559
26       13.2721
27       433.183
28       774.517
29       243.606
          ...   
46247    243.606
46248    433.183
46249    13.2721
46250    507.651
46251    783.828
46252    599.763
46253        NaN
46254    864.589
46255    333.861
46256    23.1258
46257    103.348
46258        NaN
46259    583.995
46260    228.447
46261        NaN
46262    265.357
46263    662.876
46264        NaN
46265    377.483
46266    484.098
46267    771.571
46268    333.861
46269        NaN
46270    1736.51
46271    18.8225
46272    302.155
46273    518.592
46274    333.8

In [75]:
ratio_u=sorted((value,key) for (key,value) in ratio_u_ingred.items())
ratio_u[::-1]

[(1.0, 'yuzu'),
 (1.0, 'wild duck'),
 (1.0, 'vanilla pod'),
 (1.0, 'tortilla flour'),
 (1.0, 'tongue'),
 (1.0, 'terrine'),
 (1.0, 'soda bread'),
 (1.0, 'sea trout'),
 (1.0, 'salsify'),
 (1.0, 'rye bread'),
 (1.0, 'rouille'),
 (1.0, 'roast chicken'),
 (1.0, 'risotto'),
 (1.0, 'red mullet'),
 (1.0, 'rainbow trout'),
 (1.0, 'quail'),
 (1.0, 'poussin'),
 (1.0, 'potato rosti'),
 (1.0, 'pork mince'),
 (1.0, 'plantain'),
 (1.0, 'partridge'),
 (1.0, 'nougat'),
 (1.0, 'mutton'),
 (1.0, 'mint sauce'),
 (1.0, 'mince'),
 (1.0, 'lemon sole'),
 (1.0, 'lasagne'),
 (1.0, 'lamb rump'),
 (1.0, 'lamb rack'),
 (1.0, 'ketjap manis'),
 (1.0, 'john dory'),
 (1.0, 'goose'),
 (1.0, 'globe artichoke'),
 (1.0, 'garlic bread'),
 (1.0, 'galangal'),
 (1.0, 'frangipane'),
 (1.0, 'focaccia'),
 (1.0, 'flatbread'),
 (1.0, 'english muffin'),
 (1.0, 'duxelles'),
 (1.0, 'dumpling'),
 (1.0, 'dried chilli'),
 (1.0, 'digestive biscuit'),
 (1.0, 'cumberland sauce'),
 (1.0, 'crudites'),
 (1.0, 'crostini'),
 (1.0, 'cress'),
 (1

In [ ]:
print('There are : ',len(list(ingredients_frame)), 'ingredients')
ingred_used={}
for i in list(ingredients_frame):
    ingred_used[i]=sum(ingredients_frame[i].value_counts())

In [ ]:
occu=sorted((value,key) for (key,value) in ingred_used.items())
occu[::-1]

### create the linear regression

In [83]:
a = ingredients_frame.mean(axis=0)
print(a[:30])

ale                  746.058824
almond milk          366.843750
amaranth             491.333333
amaretto              98.190476
apple chutney        237.000000
asafoetida             0.360000
baked beans          237.000000
barley               286.560241
beef consomme        402.120000
beef dripping        190.875000
beef mince           750.000000
beef oxtail          906.000000
beef shin            453.000000
boar                 453.000000
bonito               180.697143
borage                30.000000
borlotti bean        453.000000
brazil nut           379.875000
bulgur               281.068966
bulgur wheat         239.307692
calasparra rice      711.000000
calvados             185.250000
candied peel         133.000000
cassava              232.000000
caul fat             453.000000
chanterelle          642.600000
cherry brandy        126.000000
chickpea flour       221.846154
chilli powder         15.000000
chocolate biscuit    237.000000
dtype: float64


In [46]:
ingredients_frame['elderberry'].idxmax()

42577

In [136]:
from sklearn.linear_model import LinearRegression
nutri_data=raw_data[['kcal_total','kcal_carb','kcal_fat','kcal_protein','mg_sodium','mg_cholesterol']].copy()
a = nutri_data.replace('?',np.nan)
a.dropna(axis=0,how='any',inplace=True)  #drop line with NA
ingredients_frame=test_ingredients_frame
ingredients_frame['Total'] = a['kcal_total']
ingredients_frame['Carbohyd'] = a['kcal_fat']
ingredients_frame['fat'] = a['kcal_fat']
ingredients_frame['protein'] = a['kcal_protein']
ingredients_frame['mg_sodium'] = a['mg_sodium']
ingredients_frame['mg_cholestrol'] = a['mg_cholesterol']

display(ingredients_frame.head())

,acidulated water,acorn squash,agave,ale,aleppo pepper,allspice,almond,almond extract,almond milk,amaranth,...,yoghurt,yogurt,yuzu,zest,zucchini,Total,Carbohyd,protein,mg_sodium,mg_cholestrol
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,257.1,7.2,21.2,95.4,2.5
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,237,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [138]:
Train_set = ingredients_frame.dropna(subset=['Total'])

In [141]:
from sklearn.metrics import mean_absolute_error

X = Train_set.drop(['Total','Carbohyd','fat','protein','mg_sodium','mg_cholestrol'],axis=1)
X = X.replace(np.nan,0)
Y = Train_set['Total'] 
print(X.shape)
X_train = X[:20000]
Y_train = Y[:20000]

X_test = X[20000:]
Y_test = Y[20000:]

line_reg = LinearRegression(fit_intercept=False)
line_reg.fit(X_train ,Y_train)
print(line_reg.coef_)
line_reg.coef_[np.where(line_reg.coef_ < 0)] = 0
Y_predict = line_reg.predict(X_test)

print(mean_absolute_error(Y_test,Y_predict))

(23859, 812)
[ -8.09515666e-12   1.61934879e-01  -5.54883822e-03  -8.25821321e-02
   2.32639518e+00  -1.72043792e+00   2.69268759e-01   3.13170709e+00
   8.33129125e-01   3.11281307e-01   1.67720256e+00   1.60804240e+00
   1.79251881e-01  -3.13858578e+00   1.73753818e-01   8.71353830e-01
   6.22874927e-02   1.95540841e-01   2.41289382e-01  -1.36497754e-01
   1.03190823e-01   4.48408192e-01   2.46106090e-01   9.98473795e-02
  -1.10448905e+02   3.48506800e-01  -8.64286946e+01   1.56432983e-01
   2.06577052e-01  -1.51553037e+02  -6.63599614e-02   2.04899248e-01
   2.40606003e+00   3.91870462e-01   4.06245443e-01  -1.51270548e+02
   1.29204195e-01  -3.54891327e-02   3.07632884e-01  -9.97612743e-03
   2.81955312e-01   1.01146371e-01  -7.13070227e-01   1.00420117e-10
  -1.07401432e+00   7.10834591e-02   7.19962701e-02   1.61111836e-01
   1.87193377e+00  -4.81421617e+02   3.18472878e-01   9.42305400e-10
  -2.66297457e-02   1.15543149e-01   1.13349050e+00   4.87801692e-02
  -3.15408518e-01   1

In [143]:
print(len(line_reg.coef_))

812
